In [397]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  r2_score

import lightgbm as lgb
from subprocess import check_output
print(check_output(['ls','input']).decode('utf8'))

sample_submission.csv
test.csv
train.csv



In [343]:
train = pd.read_csv('input/train.csv', parse_dates=['start_date','creation_date','sell_date'])
test = pd.read_csv('input/test.csv', parse_dates=['start_date','creation_date','sell_date'])

print('Train data size: {} Test data size: {}'.format(train.shape, test.shape))

Train data size: (9366, 18) Test data size: (4801, 17)


In [344]:
train['desk_id'].fillna('DSK00000099', inplace=True)
test['desk_id'].fillna('DSK00000099', inplace=True)

train['desk_id']=train.desk_id.apply(lambda x : x[-4:]).astype(np.int16)
test['desk_id']=test.desk_id.apply(lambda x : x[-4:]).astype(np.int16)

In [345]:
le = LabelEncoder()
le.fit(train['office_id'])
train['office_id'] = le.transform(train['office_id'])
test['office_id'] = le.transform(test['office_id'])

In [346]:
le.fit(train['pf_category'])
train['pf_category'] = le.transform(train['pf_category'])
test['pf_category'] = le.transform(test['pf_category'])

In [347]:
train['start_creation'] = np.abs(train.start_date - train.creation_date).dt.days
test['start_creation'] = np.abs(test.start_date - test.creation_date).dt.days

In [348]:
train['start_sell'] = np.abs(train.start_date - train.sell_date).dt.days
test['start_sell'] = np.abs(test.start_date - test.sell_date).dt.days

In [349]:
#train['creation_sell'] = np.abs(train.creation_date - train.sell_date).dt.days
#test['creation_sell'] = np.abs(test.creation_date - test.sell_date).dt.days

In [350]:
train['sold'].fillna(68552000.0, inplace= True)
train['bought'].fillna(68427270.0, inplace= True)

In [351]:
train['dff_sold_bought'] = np.abs(train.sold - train.bought)
test['dff_sold_bought'] = np.abs(test.sold - test.bought)

In [352]:
#train['gain_loss'] = train.dff_sold_bought * train.euribor_rate 
#test['gain_loss'] = test.dff_sold_bought * test.euribor_rate 

In [353]:
train.drop(['start_date','creation_date', 'sell_date'], inplace=True, axis=1)
test.drop(['start_date','creation_date', 'sell_date'], inplace=True, axis=1)

In [354]:
le.fit(train['country_code'])
train['country_code'] = le.transform(train['country_code'])
test['country_code'] = le.transform(test['country_code'])

In [355]:
le.fit(train['currency'])
train['currency'] = le.transform(train['currency'])
test['currency'] = le.transform(test['currency'])

In [356]:
train['libor_rate'].fillna(0.0, inplace= True)
test['libor_rate'].fillna(0.0, inplace=True)

In [357]:
train['indicator_code'].fillna(False, inplace= True)
test['indicator_code'].fillna(False, inplace= True)

le.fit(train['indicator_code'])
train['indicator_code'] = le.transform(train['indicator_code'])
test['indicator_code'] = le.transform(test['indicator_code'])

In [358]:
le.fit(train['type'])
train['type'] = le.transform(train['type'])
test['type'] = le.transform(test['type'])

In [359]:
train['hedge_value'].fillna(False, inplace=True)
test['hedge_value'].fillna(False, inplace=True)

train['hedge_value'] = train['hedge_value'].map({False:0, True:1})
test['hedge_value'] = test['hedge_value'].map({False:0, True:1})

In [360]:
train['status'].fillna(False, inplace= True)
test['status'].fillna(False, inplace= True)

train['status'] = train['status'].map({False:0, True:1})
train['status'] = train['status'].map({False:0, True:1})

In [361]:
retn = train['return']
test_portfolio_id = test.portfolio_id.values

train.drop(['portfolio_id','bought', 'type', 'return'], inplace=True, axis=1)
test.drop(['portfolio_id','bought', 'type'], inplace=True, axis=1)

In [398]:
def r2(pred, drain):
    y = list(lgb_train.get_label())
    score = r2_score(y, pred)
    return 'r2_score', score, True
    

In [411]:
params = {'learning_rate': 0.02, 
          'num_leaves':78,
          'min_data_in_leaf': 160,
          'max_depth': 8, 
          'colsample_bytree': 0.522,
          'boosting': 'gbdt', 
          'application': 'regression', 
          'metric': 'mae',  
          'verbosity': -1,
          'seed': 32}

In [412]:
lgb_train = lgb.Dataset(train, label=retn)

In [413]:
lgb_cv = lgb.cv(params, lgb_train, num_boost_round=10000, nfold= 5, stratified=False,
                 verbose_eval=20, early_stopping_rounds=40)

[20]	cv_agg's l1: 0.0100323 + 9.89888e-05
[40]	cv_agg's l1: 0.00712273 + 9.05956e-05
[60]	cv_agg's l1: 0.00530528 + 9.19266e-05
[80]	cv_agg's l1: 0.00409053 + 0.00010304
[100]	cv_agg's l1: 0.00323901 + 0.000115162
[120]	cv_agg's l1: 0.00269274 + 0.000119308
[140]	cv_agg's l1: 0.00231584 + 0.000126654
[160]	cv_agg's l1: 0.00209182 + 0.000124037
[180]	cv_agg's l1: 0.00194898 + 0.000124315
[200]	cv_agg's l1: 0.001861 + 0.00012396
[220]	cv_agg's l1: 0.00181583 + 0.000123082
[240]	cv_agg's l1: 0.00177017 + 0.00012234
[260]	cv_agg's l1: 0.0017322 + 0.000119701
[280]	cv_agg's l1: 0.00170357 + 0.000119134
[300]	cv_agg's l1: 0.00168107 + 0.000117718
[320]	cv_agg's l1: 0.00166034 + 0.000118601
[340]	cv_agg's l1: 0.00163825 + 0.000118427
[360]	cv_agg's l1: 0.00161529 + 0.00011633
[380]	cv_agg's l1: 0.0016011 + 0.000118725
[400]	cv_agg's l1: 0.00158459 + 0.000116977
[420]	cv_agg's l1: 0.00156727 + 0.000119841
[440]	cv_agg's l1: 0.00155278 + 0.000118986
[460]	cv_agg's l1: 0.00154525 + 0.000118581
[

In [414]:
nround = lgb_cv['l1-mean'].index(np.min(lgb_cv['l1-mean']))
print(nround)

1606


In [415]:
model = lgb.train(params, lgb_train, num_boost_round=nround)

In [416]:
r2_score(retn, model.predict(train))

0.85386694798917262

In [418]:
pred = model.predict(test)

In [419]:
sub = pd.DataFrame({'portfolio_id':test_portfolio_id, 'return':pred})
sub.head(5)

portfolio_id    return
0   PF00001001  0.028723
1   PF00001004  0.033495
2   PF00001009  0.025105
3   PF00001013  0.027274
4   PF00001014  0.023718

In [420]:
sub.to_csv('lgb_v1.csv', index=False)